In [1]:
import requests
from datetime import datetime, timedelta
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Dict, List

In [3]:
class AirQualityDB:
    def __init__(self, db_name: str = "air_quality.db"):
        #Initilizes database and creates tables if they don't exist using below function
        self.conn = sqlite3.connect(db_name)
        self.create_tables()
    
    def create_tables(self):
        with self.conn:
            # Create locations table
            self.conn.execute("""
                CREATE TABLE IF NOT EXISTS locations (
                    location_id INTEGER PRIMARY KEY,
                    city TEXT,
                    name TEXT,
                    latitude REAL,
                    longitude REAL
                )
            """)
            
            # Create measurements table
            self.conn.execute("""
                CREATE TABLE IF NOT EXISTS measurements (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    location_id INTEGER,
                    parameter TEXT,
                    value REAL,
                    unit TEXT,
                    timestamp DATETIME,
                    latitude REAL,
                    longitude REAL,
                    FOREIGN KEY (location_id) REFERENCES locations (location_id)
                )
            """)
    
    def insert_location(self, location_id: int, city: str, name: str, latitude: float, longitude: float):
        #Insert/Update Locations into SQl db
        with self.conn:
                self.conn.execute("""
                    INSERT OR REPLACE INTO locations 
                    (location_id, city, name, latitude, longitude)
                    VALUES (?, ?, ?, ?, ?)
                """, (location_id, city, name, latitude, longitude))
    
    def insert_measurements(self, measurements: List[Dict]):
        #Inserts measurements, include locationID, paramters with their respective values and units, date and time, and coordinates\s 
        with self.conn:
            for measurement in measurements:
                    self.conn.execute("""
                        INSERT INTO measurements 
                        (location_id, parameter, value, unit, timestamp, latitude, longitude)
                        VALUES (?, ?, ?, ?, ?, ?, ?)
                    """, (
                        measurement.get('locationId'),
                        measurement.get('parameter'),
                        measurement.get('value'),
                        measurement.get('unit'),
                        measurement.get('date', {}).get('utc'),
                        measurement.get('coordinates', {}).get('latitude'),
                        measurement.get('coordinates', {}).get('longitude')
                    ))

In [4]:
def fetch_measurements(location_id: int, api_key: str, 
                      date_from: Optional[str] = None,
                      date_to: Optional[str] = None,
                      parameter: Optional[str] = None) -> Optional[Dict]:
    #Fetch mehtod accessing the OpenAQ API, fetches the raw JSON data from the API and parses into a python dictonary 
    base_url = "https://api.openaq.org/v2/measurements"
    
    params = {
        "location_id": location_id,
        "limit": 1000,
        "date_from": date_from or (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d"),
        "date_to": date_to or datetime.now().strftime("%Y-%m-%d")
    }
    
    if parameter:
        params["parameter"] = parameter
    
    headers = {"X-API-Key": api_key}
    
    response = requests.get(base_url, params=params, headers=headers)
    if response.status_code == 200:
        return response.json()
    print("Failed to fetch data!")
    return None

In [5]:
class AirQualityAnalyzer:
    def __init__(self, db_connection: sqlite3.Connection):
        self.conn = db_connection

    def get_pm25_extremes(self, city: str) -> pd.DataFrame:
        #Queries on the given city, return in a daily avg, daily min, and daily max, then utilizes rank to order by average values ASC, and order by average values DSC  
        query = """
            SELECT 
                l.name as location_name,
                DATE(m.timestamp) as date,
                AVG(m.value) as daily_avg,
                MIN(m.value) as daily_min,
                MAX(m.value) as daily_max,
                RANK() OVER (PARTITION BY l.name ORDER BY AVG(m.value) DESC) as highest_rank,
                RANK() OVER (PARTITION BY l.name ORDER BY AVG(m.value) ASC) as lowest_rank
            FROM measurements m
            JOIN locations l ON m.location_id = l.location_id
            WHERE m.parameter = 'pm25' 
            AND l.city = ?
            GROUP BY l.name, DATE(m.timestamp)
            ORDER BY l.name, date
        """
        return pd.read_sql_query(query, self.conn, params=(city,))

    def get_city_pm25_trends(self, city: str) -> pd.DataFrame:
        #Returns measurement values ordered by timestampe, this is utilized for time-wise plotting 
        query = """
            SELECT 
                m.timestamp,
                m.value,
                l.name as location_name
            FROM measurements m
            JOIN locations l ON m.location_id = l.location_id
            WHERE m.parameter = 'pm25'
            AND l.city = ?
            ORDER BY m.timestamp
        """
        return pd.read_sql_query(query, self.conn, params=(city,))

    def plot_city_pm25_trends(self, city: str):
        #Calls get_city_pm25_trends and plots trends over time for all locations in a city 
        data = self.get_city_pm25_trends(city)
        if data.empty:
            print(f"No PM2.5 data found for {city}")
            return None
            
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        
        plt.figure(figsize=(15, 7))
        
        # Plot each location with a different color
        for location in data['location_name'].unique():
            location_data = data[data['location_name'] == location]
            if not location_data.empty:
                plt.plot(location_data['timestamp'], location_data['value'], 
                        alpha=0.4, label=f'{location} (Raw)')
                
                # Add rolling average
                rolling_mean = location_data['value'].rolling(window=24).mean()
                plt.plot(location_data['timestamp'], rolling_mean, 
                        alpha=1.0, label=f'{location} (24h Avg)')
        
        plt.title(f'PM2.5 Trends in {city}')
        plt.xlabel('Date')
        plt.ylabel('PM2.5 (µg/m³)')
        plt.grid(True, alpha=0.3)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        return plt

    def compare_ozone_levels(self, city1: str, city2: str) -> pd.DataFrame:
        #Compares daily average ozone levels between two cities
        query = """
            SELECT 
                m1.date,
                m1.daily_avg_o3 as city1_o3,
                m2.daily_avg_o3 as city2_o3
            FROM (
                SELECT 
                    DATE(m.timestamp) as date,
                    AVG(m.value) as daily_avg_o3
                FROM measurements m
                JOIN locations l ON m.location_id = l.location_id
                WHERE m.parameter = 'o3' AND l.city = ?
                GROUP BY DATE(m.timestamp)
            ) m1
            LEFT JOIN (
                SELECT 
                    DATE(m.timestamp) as date,
                    AVG(m.value) as daily_avg_o3
                FROM measurements m
                JOIN locations l ON m.location_id = l.location_id
                WHERE m.parameter = 'o3' AND l.city = ?
                GROUP BY DATE(m.timestamp)
            ) m2 ON m1.date = m2.date
            ORDER BY m1.date
    """
        return pd.read_sql_query(query, self.conn, params=(city1, city2))

    def get_city_ozone_levels(self, city: str) -> pd.DataFrame:
        #Gets ozone levels for each loction within a city, ie NYC(Manhattan, Brooklyn, Queens)
        query = """
            SELECT 
                l.name as location_name,
                DATE(m.timestamp) as date,
                AVG(m.value) as daily_avg_o3
            FROM measurements m
            JOIN locations l ON m.location_id = l.location_id
            WHERE m.parameter = 'o3'
            AND l.city = ?
            GROUP BY l.name, DATE(m.timestamp)
            ORDER BY l.name, date
        """
        return pd.read_sql_query(query, self.conn, params=(city,))

    def plot_cities_ozone_comparison(self, city1: str, city2: str):
        #plots comparison made in above function 
        data = self.compare_ozone_levels(city1, city2)
        if data.empty:
            print(f"No ozone data found for comparison between {city1} and {city2}")
            return None
            
        plt.figure(figsize=(15, 7))
        
        plt.plot(data['date'], data['city1_o3'], 
                alpha=0.6, label=f'{city1} Average', linewidth=2)
        plt.plot(data['date'], data['city2_o3'], 
                alpha=0.6, label=f'{city2} Average', linewidth=2)
        
        plt.title(f'Ozone Level Comparison: {city1} vs {city2}')
        plt.xlabel('Date')
        plt.ylabel('O₃ (ppb)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        return plt

In [6]:
def main():
    api_key = "4557293704f9ea4cfe0e78910aec656f78e52cf707dc5728c677556cccb785ac"
    
    city_locations = {
        "NewYork": [
            (625, "New York - Manhattan"),
            (648, "New York - Brooklyn"),
            (631, "New York - Queens")
        ],
        #Boston required 2 more additions over New York due to either measurements not being recorded which they were stated to record or machine malfunction, only Roxbury, Fort Hill and Chinatown truly recorded PM2.5 levels 
        "Boston": [
            (384, "Boston - Chinatown"),
            (521, "Boston - Kenmore"),
            (448, "Boston - Roxbury"),
            (452, "Boston - Von Hillern"),
            (2117520, "Boston- Roxbury, Fort Hill")
        ]
    }
    # Initialize database and fectch data for the past month 
    db = AirQualityDB()
    date_from = "2024-11-16"
    date_to = "2024-12-16"
    print(f"Fetching data from {date_from} to {date_to}")
    
    # store the fetched data for each city and location combo 
    for city, locations in city_locations.items():
        print(f"\nProcessing {city} locations:")
        for location_id, location_name in locations:
            print(f"Fetching data for {location_name} (ID: {location_id})")
            
            data = fetch_measurements(
                location_id,
                api_key,
                date_from=date_from,
                date_to=date_to
            )
            
            if data and 'results' in data:
                print(f"Found {len(data['results'])} measurements")
                
                # Store location metadata
                first_result = data['results'][0] if data['results'] else None
                if first_result:
                    coords = first_result.get('coordinates', {})
                    db.insert_location(
                        location_id,
                        city,
                        location_name,
                        coords.get('latitude'),
                        coords.get('longitude')
                    )
                
                # Store measurements using the insert_measurements function 
                db.insert_measurements(data['results'])
            else:
                print(f"No data found or error in API response")
    
    # Initialize analyzer from class AirQualityAnalyzer
    analyzer = AirQualityAnalyzer(db.conn)
    
    # Try to create and save the plot
    for city in city_locations.keys():
        print(f"\nAnalyzing {city} data:")
            
        # Get PM2.5 extremes
        pm25_extremes = analyzer.get_pm25_extremes(city)
        if not pm25_extremes.empty:
            print(f"\nPM2.5 Extreme Periods in {city}:")
            print("\nHighest PM2.5 Days:")
            print(pm25_extremes.nsmallest(5, 'highest_rank')[['date', 'daily_avg']])
            print("\nLowest PM2.5 Days:")
            print(pm25_extremes.nsmallest(5, 'lowest_rank')[['date', 'daily_avg']])
            
        # Generate PM2.5 trends plot
        pm25_plot = analyzer.plot_city_pm25_trends(city)
        if pm25_plot:
            pm25_plot.savefig(f'{city.lower()}_pm25_trends.png', bbox_inches='tight')
            plt.close()

    # Compare ozone levels between cities
    ozone_plot = analyzer.plot_cities_ozone_comparison("NewYork", "Boston")
    if ozone_plot:
        ozone_plot.savefig('ozone_cities_comparison.png', bbox_inches='tight')
        plt.close()
    
    db.conn.close()

In [7]:
if __name__ == "__main__":
    main()

Fetching data from 2024-11-16 to 2024-12-16

Processing NewYork locations:
Fetching data for New York - Manhattan (ID: 625)
Found 717 measurements
Fetching data for New York - Brooklyn (ID: 648)
Found 638 measurements
Fetching data for New York - Queens (ID: 631)
Found 1000 measurements

Processing Boston locations:
Fetching data for Boston - Chinatown (ID: 384)
Found 1000 measurements
Fetching data for Boston - Kenmore (ID: 521)
Found 1000 measurements
Fetching data for Boston - Roxbury (ID: 448)
Found 1000 measurements
Fetching data for Boston - Von Hillern (ID: 452)
Found 1000 measurements
Fetching data for Boston- Roxbury, Fort Hill (ID: 2117520)
Found 1000 measurements

Analyzing NewYork data:

PM2.5 Extreme Periods in NewYork:

Highest PM2.5 Days:
          date  daily_avg
21  2024-12-10  19.537500
29  2024-11-17  13.308333
67  2024-12-10  14.541667
20  2024-12-09  12.166667
51  2024-12-09  13.258333

Lowest PM2.5 Days:
          date  daily_avg
8   2024-11-24   2.141667
33  2024